In [1]:
import torch
import numpy as np
f = torch.load('/home/ivb/Deploy/cds-mmdetection/checkpoints/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c.pth')

In [2]:
f.keys()

dict_keys(['meta', 'state_dict'])

In [3]:
f['meta'].keys()

dict_keys(['mmdet_version', 'config', 'epoch', 'iter', 'mmcv_version', 'time'])

In [4]:
f['meta']['config']

"# model settings\nmodel = dict(\n    type='HybridTaskCascade',\n    num_stages=3,\n    pretrained='open-mmlab://resnext101_64x4d',\n    interleaved=True,\n    mask_info_flow=True,\n    backbone=dict(\n        type='ResNeXt',\n        depth=101,\n        groups=64,\n        base_width=4,\n        num_stages=4,\n        out_indices=(0, 1, 2, 3),\n        frozen_stages=1,\n        style='pytorch',\n        dcn=dict(\n            modulated=False,\n            groups=64,\n            deformable_groups=1,\n            fallback_on_stride=False),\n        stage_with_dcn=(False, True, True, True)),\n    neck=dict(\n        type='FPN',\n        in_channels=[256, 512, 1024, 2048],\n        out_channels=256,\n        num_outs=5),\n    rpn_head=dict(\n        type='RPNHead',\n        in_channels=256,\n        feat_channels=256,\n        anchor_scales=[8],\n        anchor_ratios=[0.5, 1.0, 2.0],\n        anchor_strides=[4, 8, 16, 32, 64],\n        target_means=[.0, .0, .0, .0],\n        target_stds

In [5]:
f['state_dict'].keys()

odict_keys(['backbone.conv1.weight', 'backbone.bn1.weight', 'backbone.bn1.bias', 'backbone.bn1.running_mean', 'backbone.bn1.running_var', 'backbone.bn1.num_batches_tracked', 'backbone.layer1.0.conv1.weight', 'backbone.layer1.0.bn1.weight', 'backbone.layer1.0.bn1.bias', 'backbone.layer1.0.bn1.running_mean', 'backbone.layer1.0.bn1.running_var', 'backbone.layer1.0.bn1.num_batches_tracked', 'backbone.layer1.0.conv2.weight', 'backbone.layer1.0.bn2.weight', 'backbone.layer1.0.bn2.bias', 'backbone.layer1.0.bn2.running_mean', 'backbone.layer1.0.bn2.running_var', 'backbone.layer1.0.bn2.num_batches_tracked', 'backbone.layer1.0.conv3.weight', 'backbone.layer1.0.bn3.weight', 'backbone.layer1.0.bn3.bias', 'backbone.layer1.0.bn3.running_mean', 'backbone.layer1.0.bn3.running_var', 'backbone.layer1.0.bn3.num_batches_tracked', 'backbone.layer1.0.downsample.0.weight', 'backbone.layer1.0.downsample.1.weight', 'backbone.layer1.0.downsample.1.bias', 'backbone.layer1.0.downsample.1.running_mean', 'backbone.

# Change number of classes in static_dict

In [6]:
indices = [0, 1, 2, 3, 4, 6, 7, 8, 10, 12, 17, 18, 19, 20, 21, 22, 23, 24]
num_of_cls = 81
for key in f['state_dict'].keys():
    dim = list(f['state_dict'][key].shape)
    if not dim == []:
        if dim[0] % num_of_cls == 0:
            print(key, end=" ")
            cardinality = dim[0] // num_of_cls
            lst = []
            R = 0
            n = 0
            for idx in range(num_of_cls):
                t = f["state_dict"][key][idx * cardinality:(idx + 1) * cardinality]
                if idx in indices:
                    lst.append(t)
                else:
                    R += t
                    n += 1
            lst[0] = (lst[0] + R)
            f['state_dict'][key] = torch.cat(lst)
            print(dim, "->", list(f['state_dict'][key].shape))

bbox_head.0.fc_cls.weight [81, 1024] -> [18, 1024]
bbox_head.0.fc_cls.bias [81] -> [18]
bbox_head.1.fc_cls.weight [81, 1024] -> [18, 1024]
bbox_head.1.fc_cls.bias [81] -> [18]
bbox_head.2.fc_cls.weight [81, 1024] -> [18, 1024]
bbox_head.2.fc_cls.bias [81] -> [18]
mask_head.0.conv_logits.weight [81, 256, 1, 1] -> [18, 256, 1, 1]
mask_head.0.conv_logits.bias [81] -> [18]
mask_head.1.conv_logits.weight [81, 256, 1, 1] -> [18, 256, 1, 1]
mask_head.1.conv_logits.bias [81] -> [18]
mask_head.2.conv_logits.weight [81, 256, 1, 1] -> [18, 256, 1, 1]
mask_head.2.conv_logits.bias [81] -> [18]


In [10]:
del f["meta"]

In [8]:
torch.save(f, '/home/ivb/Deploy/cds-mmdetection/checkpoints/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c_road.pth')

# Change number of classes in meta

In [7]:
f['meta']['config'] = f['meta']['config'].replace("81", "18")

# Add my own classes

In [195]:
f['meta']['CLASSES'] = 'traffic_light'

In [196]:
f['meta'].keys()

dict_keys(['mmdet_version', 'config', 'epoch', 'iter', 'mmcv_version', 'time', 'CLASSES'])

# Save .pth file

In [197]:
torch.save(f, '/home/serg_t/Documents/mmdetection/trained_models/faster_rcnn_r101_fpn_1x_20181129-d1468807_tl_only.pth')
f_tl = torch.load('/home/serg_t/Documents/mmdetection/trained_models/faster_rcnn_r101_fpn_1x_20181129-d1468807_tl_only.pth')


# Test

In [198]:
f_tl['meta']['config'];

In [199]:
for key in f_tl['state_dict'].keys():
    print(f_tl['state_dict'][key].shape);

torch.Size([64, 3, 7, 7])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([])
torch.Size([256, 64, 1, 1])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([])
torch.Size([256, 64, 1, 1])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([])
torch.Size([64, 256, 1, 1])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([])
torch.Size([256, 64, 1, 1])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([])
torch.Size([64, 256, 1, 1])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])

In [200]:
my_trained = torch.load('/home/serg_t/Documents/mmdetection/cascade_rcnn_predtrained/epoch_12.pth')
my_trained['meta']['CLASSES']

'traffic_light'